In [1]:
import os
import pymysql
import pandas as pd
import numpy as np
import re
from pandas.io import sql
import MySQLdb
from sqlalchemy import create_engine

In [3]:
from pandas.io import sql
import MySQLdb

conn = MySQLdb.connect(
    host="localhost",
    port=int(3306),
    user="pythonlink",
    passwd="pythonlinkp",
    db="stock_forecast_v1",
    charset='utf8mb4'
)

data = pd.read_sql_query("SELECT * FROM delta_mergedstatements500_withtrendclass",
    conn)
data = data.drop(['Price'], axis = 1)
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Classification
0,A,2011-10-01,-639.0,-1719.0,1080.0,0.5,3.11,1034.0,2.98,-1506.0,...,188.8,328.0,542.0,2468.0,-2294.0,708.0,3284.1,3277.9,1.37,1
1,A,2012-10-01,1479.0,610.0,869.0,-0.2,2.52,-1241.0,-3.58,177.0,...,27.5,141.0,-32.0,-3660.0,1656.0,-2054.0,-1592.0,-1582.6,-0.12,0
2,A,2013-10-01,150.0,46.0,104.0,-16.1,0.95,252.0,0.89,337.0,...,-485.7,-419.0,-76.0,2118.0,-517.0,1500.0,-324.2,-320.5,-0.16,1
3,A,2014-10-01,129.0,114.0,15.0,3.5,-0.08,822.0,2.42,-1036.0,...,15.7,-185.0,-421.0,18.0,437.0,29.0,-782.3,-780.4,-1.23,1
4,A,2015-10-01,-3336.0,-2202.0,-1134.0,-3.1,-3.26,-789.0,-2.31,-8.0,...,103.1,-148.0,591.0,-170.0,-972.0,-568.0,1850.8,1823.2,2.10,0
5,A,2016-10-01,315.0,239.0,76.0,-10.5,0.55,-46.0,-0.04,249.0,...,101.2,61.0,-529.0,162.0,821.0,501.0,-524.6,-520.8,-1.67,1
6,A,2017-10-01,632.0,43.0,589.0,1.3,1.88,553.0,1.74,107.0,...,106.3,222.0,96.0,-66.0,66.0,105.0,-320.5,-316.2,0.20,1
7,A,2018-10-01,115.0,379.0,-264.0,-4.5,-0.63,-760.0,-2.32,-212.0,...,103.1,-368.0,198.0,-401.0,-595.0,-823.0,144.5,142.0,0.62,0
8,A,2019-10-01,911.0,734.0,177.0,-8.3,0.99,-1055.0,-3.31,608.0,...,23.1,755.0,-66.0,-885.0,498.0,-434.0,158.3,157.8,-0.08,1
9,AAL,2011-12-01,-1240.0,1926.0,-3166.0,1.9,-9.38,-3128.0,-9.25,-434.0,...,98.2,-1508.0,-498.0,738.0,-140.0,100.0,526.5,538.4,-4.80,0


In [4]:
# Shuffling the data to create random arrangement of data instances
data = data.sample(frac=1).reset_index(drop=True)
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Classification
0,CINF,2019-12-01,3473.0,1442.0,2031.0,-0.2,12.45,2031.0,12.45,19.0,...,1388.1,1710.0,27.0,-228.0,57.0,-144.0,3504.1,3504.1,0.14,1
1,SIRI,2015-12-01,-322.4,1154.0,-1476.3,-485.4,-0.26,-1424.6,-0.33,953.5,...,97.6,16.5,-9.1,-42.6,2.9,-48.8,1267.4,1299.4,0.01,1
2,SNA,2016-12-01,392.1,187.6,204.5,-0.1,3.62,109.5,1.94,130.1,...,56.8,67.7,68.9,-167.0,120.7,24.9,113.2,113.5,1.29,1
3,ATNI,2018-12-01,-98.3,-91.4,-6.9,0.0,0.45,11.3,0.74,-64.8,...,-11.6,-11.7,-29.8,85.0,-13.1,41.5,34.4,33.7,-2.31,1
4,DRI,2019-05-01,423.2,225.4,197.8,-0.4,1.67,208.8,1.69,176.5,...,56.2,117.4,255.9,-11.7,152.4,396.6,46.6,43.1,1.61,1
5,ABC,2017-09-01,1679.0,1743.9,-64.9,-2.0,-0.21,16.8,-0.21,-669.0,...,292.7,-1063.4,-1674.4,2671.2,-1877.9,-881.1,-530.3,-527.3,-8.04,1
6,HPQ,2013-10-01,-3092.0,-7915.0,4823.0,-39.3,2.86,6124.0,3.02,-5771.0,...,-556.8,17763.0,1037.0,650.0,-4083.0,-2396.0,-1996.3,-2051.2,0.87,1
7,SJM,2012-04-01,790.6,919.6,-129.0,-4.0,0.47,-617.8,-5.75,765.1,...,-9.5,-19.8,339.3,-843.0,390.0,-126.3,217.6,225.5,2.24,1
8,WHR,2018-12-01,-1691.0,232.0,-1923.0,-7.1,-23.36,-945.0,-17.27,809.0,...,-90.8,-533.0,-35.0,322.0,35.0,192.0,210.8,229.6,1.60,0
9,SNA,2017-12-01,525.9,188.8,337.1,-1.3,6.95,239.0,4.81,176.6,...,10.1,11.3,32.4,132.0,-140.1,29.6,-109.4,-109.2,0.53,1


In [6]:
#data.to_csv(r'C:\Users\conor\Documents\MScAIProject\MSc_AI_Project\Data\Testing\DeltaFilings_PriceTrend\MergedStatements\HyperparameterTuning\MFTrend_data.csv', index = False)